In [36]:
import polars as pl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import polars as pl
import scipy.stats as stats
import scipy.sparse as sps
import gc
import tqdm
from polimi.utils._custom import save_sparse_csr

In [37]:
from os import system, getpid, walk
from psutil import Process
from colorama import Fore, Style, init
from IPython.display import display, HTML

def PrintColor(text:str, color = Fore.BLUE, style = Style.BRIGHT):
    print(style + color + text + Style.RESET_ALL)
    
def GetMemUsage():   
    pid = getpid()
    py = Process(pid)
    memory_use = py.memory_info()[0] / 2. ** 30
    return f"RAM memory GB usage = {memory_use :.4}"

PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 2.591


In [38]:
def reduce_polars_df_memory_size(df, set_categorical=True):

    start_mem = df.estimated_size('mb')
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type in [pl.Int16, pl.Int32, pl.Int64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                df = df.with_columns(pl.col(col).cast(pl.Int8))
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                df = df.with_columns(pl.col(col).cast(pl.Int16))
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                df = df.with_columns(pl.col(col).cast(pl.Int32))
        elif col_type in [pl.UInt16, pl.UInt32, pl.UInt64]:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt8))
            elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt16))
            elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                df = df.with_columns(pl.col(col).cast(pl.UInt32))
        elif col_type == pl.Float64:
            c_min = df[col].fill_null(0).min()
            c_max = df[col].fill_null(0).max()
            if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                df = df.with_columns(pl.col(col).cast(pl.Float32))

    gc.collect()
    end_mem = df.estimated_size('mb')
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df

In [31]:
articles = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/articles.parquet')

behaviors_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/behaviors.parquet')
history_train = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/train/history.parquet')

behaviors_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/behaviors.parquet')
history_val = pl.read_parquet('/home/ubuntu/dataset/ebnerd_small/validation/history.parquet')

contrastive_vector = pl.read_parquet('/home/ubuntu/dataset/Ekstra_Bladet_contrastive_vector/contrastive_vector.parquet')

gc.collect()
PrintColor(f"\n" + GetMemUsage(), color = Fore.RED)


RAM memory GB usage = 2.231


In [32]:
articles_mapping = articles.select('article_id').with_row_index().rename({'index': 'article_index'})
articles_mapping

article_index,article_id
u32,i32
0,3001353
1,3003065
2,3012771
3,3023463
4,3032577
…,…
20733,9803492
20734,9803505
20735,9803525


In [33]:
ICM_dataframe = contrastive_vector.join(articles, on='article_id').select(['article_id','contrastive_vector']).with_columns(
        pl.col("contrastive_vector").apply(lambda lst : list(range(len(lst)))).alias("indici")      
    )\
    .explode(['contrastive_vector','indici'])\
    .rename({'indici': 'feature_id'})\
    .join(articles_mapping, on='article_id')\
    .drop('article_id')
ICM_dataframe

/tmp/ipykernel_126061/1601828010.py:2: DeprecationWarning: `apply` is deprecated. It has been renamed to `map_elements`.
  pl.col("contrastive_vector").apply(lambda lst : list(range(len(lst)))).alias("indici")
/tmp/ipykernel_126061/1601828010.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ICM_dataframe = contrastive_vector.join(articles, on='article_id').select(['article_id','contrastive_vector']).with_columns(


contrastive_vector,feature_id,article_index
f32,i64,u32
0.014536,0,0
0.013818,1,0
0.01941,2,0
-0.071472,3,0
-0.012827,4,0
…,…,…
0.005754,763,20737
0.028954,764,20737
0.004138,765,20737


In [8]:
"""
contrastive_vector = contrastive_vector.select(
    pl.exclude('contrastive_vector'), 
    *[pl.col('contrastive_vector').list.get(x).alias(f"feature_{x}") 
          for x in range(n_features)]
)


contrastive_vector
"""

'\ncontrastive_vector = contrastive_vector.select(\n    pl.exclude(\'contrastive_vector\'), \n    *[pl.col(\'contrastive_vector\').list.get(x).alias(f"feature_{x}") \n          for x in range(n_features)]\n)\n\n\ncontrastive_vector\n'

In [34]:
n_articles = ICM_dataframe.select('article_index').n_unique()
print(f'n_articles:{n_articles}')
n_features = ICM_dataframe.select('feature_id').n_unique()
print(f'num_features: {n_features}')

n_articles:20738
num_features: 768


In [35]:
ICM = sps.csr_matrix((ICM_dataframe["contrastive_vector"].to_numpy(), 
                          (ICM_dataframe["article_index"].to_numpy(), ICM_dataframe["feature_id"].to_numpy())),
                        shape = (n_articles, n_features))

ICM

<20738x768 sparse matrix of type '<class 'numpy.float32'>'
	with 15926784 stored elements in Compressed Sparse Row format>

In [40]:
save_sparse_csr(path=Path('/home/ubuntu/recsys2024/icm/recsys/small/ICM.npz'),array=ICM)

File saved at: /home/ubuntu/recsys2024/icm/recsys/small/ICM.npz


In [11]:
from RecSys_Course_AT_PoliMi.Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN.UserKNNCBFRecommender import UserKNNCBFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from RecSys_Course_AT_PoliMi.Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from RecSys_Course_AT_PoliMi.Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender


In [13]:
from polimi.utils._custom import load_sparse_csr

URM_train = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_train.npz'))
URM_validation_train = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation_train.npz'))
URM_validation = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation.npz'))
URM_validation_validation = load_sparse_csr(Path('/home/ubuntu/recsys2024/urm/recsys/small/URM_validation_validation.npz'))


File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_train.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation_train.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation.npz
File loaded at: /home/ubuntu/recsys2024/urm/recsys/small/URM_validation_validation.npz


In [14]:
ICM.shape

(20738, 768)

In [15]:
URM_train.shape

(18827, 20738)

In [16]:
URM_validation_train.shape

(18827, 20738)

In [17]:
svd = PureSVDRecommender(URM_train)
svd.fit(num_factors = 15)

PureSVDRecommender: URM Detected 3684 (19.6%) users with no interactions.
PureSVDRecommender: URM Detected 11952 (57.6%) items with no interactions.
PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 0.65 sec


In [18]:
svd_item = PureSVDItemRecommender(URM_train)
svd_item.fit(num_factors= 12, topK= 1391)

PureSVDItemRecommender: URM Detected 3684 (19.6%) users with no interactions.
PureSVDItemRecommender: URM Detected 11952 (57.6%) items with no interactions.
PureSVDItemRecommender: Computing SVD decomposition...
PureSVDItemRecommender: Computing SVD decomposition... Done!


In [19]:
rp3 = RP3betaRecommender(URM_train)
rp3.fit(topK= 797, normalize_similarity = True, alpha= 1.892317046771731, beta= 0.0699278679772357)

RP3betaRecommender: URM Detected 3684 (19.6%) users with no interactions.
RP3betaRecommender: URM Detected 11952 (57.6%) items with no interactions.
RP3betaRecommender: Similarity column 20738 (100.0%), 1503.42 column/sec. Elapsed time 13.79 sec


In [20]:
knn_icm = ItemKNNCBFRecommender(URM_train=URM_train, ICM_train=ICM)
knn_icm.fit()

ItemKNNCBFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCBFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Compute_Similarity: detected dense matrix
Similarity column 20738 (100.0%), 216.28 column/sec. Elapsed time 1.60 min


In [21]:
knn = ItemKNNCFRecommender(URM_train=URM_train)
knn.fit(similarity= 'euclidean', topK= 135, shrink= 751, normalize_avg_row= True, similarity_from_distance_mode= 'log', normalize= True)

ItemKNNCFRecommender: URM Detected 3684 (19.6%) users with no interactions.
ItemKNNCFRecommender: URM Detected 11952 (57.6%) items with no interactions.
Similarity column 20738 (100.0%), 582.78 column/sec. Elapsed time 35.58 sec


In [22]:
from RecSys_Course_AT_PoliMi.Evaluation.Evaluator import EvaluatorHoldout
evaluator_validation = EvaluatorHoldout(URM_validation_train, cutoff_list=[100])

EvaluatorHoldout: Ignoring 3684 (19.6%) Users that have less than 1 test interactions


In [23]:
result_df, _ = evaluator_validation.evaluateRecommender(knn_icm)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 17.93 sec. Users per second: 844


0.011670834446676565

In [24]:
result_df, _ = evaluator_validation.evaluateRecommender(knn)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 17.03 sec. Users per second: 889


0.0027220061292659616

In [25]:
result_df, _ = evaluator_validation.evaluateRecommender(svd)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 16.34 sec. Users per second: 927


0.0035676075287137273

In [26]:
result_df, _ = evaluator_validation.evaluateRecommender(svd_item)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 47.74 sec. Users per second: 317


0.004990688308022659

In [27]:
result_df, _ = evaluator_validation.evaluateRecommender(rp3)
result_df.loc[100]["NDCG"]

EvaluatorHoldout: Processed 15143 (100.0%) in 17.83 sec. Users per second: 849


0.005401973967436028